# Narrative + QC Testing

In [1]:
import dill
import copy

from generation import reset_to_stage, generate_narrative, quality_check
from llm_utils import GenerationContext

with open('test_examples_v3.pkl', 'rb') as f:
    all_people = dill.load(f)

print(f"Loaded {len(all_people)} people")

Loaded 53 people


In [2]:
# List people
for i, p in enumerate(all_people):
    loc = p.location.country if p.location else p.region
    name = p.name[:25] if p.name else "(no name)"
    print(f"{i:3d}. {name:25s} | {p.birth_year_str:12s} | age {str(p.age_at_death):6s} | {loc}")

  0. Ptolemaios                | 300 BC       | age 51     | Turkey
  1. Shashikala                | 558 AD       | age 69     | India
  2. Okiku (お菊)                | 1622 AD      | age 69     | Japan
  3. Amina                     | 1064 AD      | age 0      | Afghanistan
  4. Tanem                     | 6552 BC      | age 74     | United States
  5. (no name)                 | 1137 AD      | age 0      | China
  6. Ambiknos                  | 628 BC       | age 0      | Spain
  7. (no name)                 | 1707 BC      | age 0      | India
  8. Mumbi                     | 318 AD       | age 34     | Kenya
  9. 邓来娣                       | 1896 AD      | age 84     | China
 10. Samira                    | 30674 BC     | age 17     | Asia
 11. Uma                       | 873 AD       | age 1      | India
 12. 刘少娥                       | 158 BC       | age 71     | China
 13. Isabeau                   | 1609 AD      | age 0      | France
 14. Theresa Walsh             | 1913 AD      |

In [3]:
# Pick a person, strip to pre-narrative, regenerate narrative + QC
test_idx = 0
person = copy.deepcopy(all_people[test_idx])
reset_to_stage(person, 'narrative')

ctx = GenerationContext(model='gpt-5.2', quiet=False, show_cost=True)

generate_narrative(person, ctx)
narrative_before_qc = person.narrative

issues = quality_check(person, ctx)
narrative_after_qc = person.narrative

ctx.finish()

print("=== NARRATIVE (before QC) ===")
print(narrative_before_qc)
print()
print("=== QC ISSUES ===")
for issue in issues:
    print(f"  - {issue}")
print()
print("=== NARRATIVE (after QC) ===")
print(narrative_after_qc)

Generating narrative (adult)...
  Generated 1347 words
Running quality check on narrative...
  Fixed 7 issues

=== Cost Summary: gpt-5.2 ===
Requests: 2
Input tokens: 14,255
  Cached: 9,856 (69.1%)
Output tokens: 3,765
Total cost: $0.0444
Avg per request: 7128 in, 1882 out
=== NARRATIVE (before QC) ===
Ptolemaios was born in a small village on the high ground south of the Halys basin, where pine and oak covered the ridges and the fields sat in pockets of thin soil. The older men still spoke the local tongue at the threshing floor and the water trough. Traders and officials spoke Greek. The region answered to whichever Hellenistic power held the roads and the tax registers that year.

His father, Dadas, kept a small plot and a few animals and went away for seasonal work. He hauled timber down to an estate yard when orders came, or burned charcoal in the hills when there was demand. He returned with coin and blisters and news from market towns. His mother, Timarete, ran the house. She gr

In [9]:
# New QC prompt - focused on anachronisms and errors, minimal prose changes

QC_PROMPT_V2 = """Review this narrative for issues and provide a corrected version.

Person details:
{person_data}

Original narrative:
{narrative}

CHECK FOR:

1. ANACHRONISMS: Technology, terminology, or social structures that couldn't exist in this time/place.

2. HISTORICAL ERRORS: Wrong dates, implausible events, cultural contradictions for this specific era and region.

3. MISSING SOCIAL CONTEXT: Places where adding a concrete detail about who arranged something or how a process worked would clarify.

4. EGREGIOUS PROSE ONLY:
   - Actual metaphors, similes, personification
   - Philosophical commentary or moralizing
   - Lines that exist to sound wise or poignant
   - Non-sequiturs

DO NOT CHANGE:
- Sentence structure or rhythm
- Sensory details (smells, textures, weather, physical descriptions)
- Specific details—specificity is good, don't make things more generic
- Gregorian calendar dates (intentional)
- Modern place names (intentional, for reader orientation)
- Metadata in person_data (map URLs, etc.)

Make minimal surgical edits. Preserve the original prose style.

Return as JSON:
{{
    "issues_found": ["list of issues"],
    "revised_narrative": "the corrected narrative"
}}"""

QC_CHECKS_ERA_V2 = {
    'Holocene': """
Check for:
1. ANACHRONISMS: Things that couldn't exist in this time/place
2. HISTORICAL IMPOSSIBILITIES: Events, technologies, or social situations that contradict history
3. CULTURAL CONTRADICTIONS: Behaviors that wouldn't occur in this culture/period
""",
    'Paleolithic': """
Check for:
1. ANACHRONISMS: Agriculture, metal, pottery before it existed, domesticated animals other than dogs
2. OVERSPECIFICITY: Claims about beliefs, rituals, or language that we can't know
"""
}

In [10]:
from generation import call_with_retry

def quality_check_v2(person, ctx):
    """QC with new lighter-touch prompt."""
    qc_prompt = QC_CHECKS_ERA_V2[person.era] + QC_PROMPT_V2.format(
        person_data=person.to_prompt_string(),
        narrative=person.narrative
    )
    
    ctx.log("Running quality check (v2)...")
    
    messages = [{"role": "user", "content": qc_prompt}]
    result, _ = call_with_retry(ctx, messages)
    
    if result:
        issues = result.get('issues_found', [])
        person.narrative = result.get('revised_narrative', person.narrative)
        ctx.log(f"  Fixed {len(issues)} issues")
        return issues
    else:
        ctx.log("  QC failed, keeping original")
        return []

In [11]:
# Test new QC prompt
test_idx = 0
person = copy.deepcopy(all_people[test_idx])
reset_to_stage(person, 'narrative')

ctx = GenerationContext(model='gpt-5.2', quiet=False, show_cost=True)

generate_narrative(person, ctx)
narrative_before_qc = person.narrative

issues = quality_check_v2(person, ctx)
narrative_after_qc = person.narrative

ctx.finish()

print("=== NARRATIVE (before QC) ===")
print(narrative_before_qc)
print()
print("=== QC ISSUES ===")
for issue in issues:
    print(f"  - {issue}")
print()
print("=== NARRATIVE (after QC) ===")
print(narrative_after_qc)

Generating narrative (adult)...
  Generated 1298 words
Running quality check (v2)...
  Fixed 8 issues

=== Cost Summary: gpt-5.2 ===
Requests: 2
Input tokens: 14,073
  Cached: 10,880 (77.3%)
Output tokens: 3,742
Total cost: $0.0428
Avg per request: 7036 in, 1871 out
=== NARRATIVE (before QC) ===
Ptolemaios was born in a timbered upland village in the interior of north-central Anatolia, where the old local speech still filled kitchens and courtyards and Greek carried weight at shrines, markets, and the desk of any official. Soldiers and tax men came and went under shifting Hellenistic rulers, and villages learned to answer to whoever held the roads that season.

His father, Dadas, kept a small plot and a few animals, but he also left for stretches of the year to take paid work hauling loads and cutting wood for charcoal on an estate that sat lower on the slopes. The household was a single roof: Dadas, his wife Nana, and their children. Three girls came first. The eldest, Aba, learned ea

In [12]:
# Test new QC prompt
test_idx = 1
person = copy.deepcopy(all_people[test_idx])
reset_to_stage(person, 'narrative')

ctx = GenerationContext(model='gpt-5.2', quiet=False, show_cost=True)

generate_narrative(person, ctx)
narrative_before_qc = person.narrative

issues = quality_check_v2(person, ctx)
narrative_after_qc = person.narrative

ctx.finish()

print("=== NARRATIVE (before QC) ===")
print(narrative_before_qc)
print()
print("=== QC ISSUES ===")
for issue in issues:
    print(f"  - {issue}")
print()
print("=== NARRATIVE (after QC) ===")
print(narrative_after_qc)

Generating narrative (adult)...
  Generated 1415 words
Running quality check (v2)...
  Fixed 4 issues

=== Cost Summary: gpt-5.2 ===
Requests: 2
Input tokens: 17,003
  Cached: 12,928 (76.0%)
Output tokens: 3,887
Total cost: $0.0456
Avg per request: 8502 in, 1944 out
=== NARRATIVE (before QC) ===
Shashikala was born in 558 in a small farming hamlet in the forested uplands west of the great coastal plain, where Eastern Indo‑Aryan speech was thick with local turns and borrowed words. Chiefs and overlords changed names and banners beyond the trees, but in her village rule showed up as demands: grain measured out, men called for carrying loads, cattle counted when officials came through.

Her father, Vishnugupta, plowed his own fields and watched the boundaries. He kept a pair of oxen and a few cows and argued with neighbors about grazing paths. Her mother, Ganga, ran the house. She rose before light, stoked the cooking fire, and set Shashikala to small tasks early. Sorting lentils. Watchin

In [13]:
# Test new QC prompt
test_idx = 2
person = copy.deepcopy(all_people[test_idx])
reset_to_stage(person, 'narrative')

ctx = GenerationContext(model='gpt-5.2', quiet=False, show_cost=True)

generate_narrative(person, ctx)
narrative_before_qc = person.narrative

issues = quality_check_v2(person, ctx)
narrative_after_qc = person.narrative

ctx.finish()

print("=== NARRATIVE (before QC) ===")
print(narrative_before_qc)
print()
print("=== QC ISSUES ===")
for issue in issues:
    print(f"  - {issue}")
print()
print("=== NARRATIVE (after QC) ===")
print(narrative_after_qc)

Generating narrative (adult)...
  Generated 1282 words
Running quality check (v2)...
  Fixed 9 issues

=== Cost Summary: gpt-5.2 ===
Requests: 2
Input tokens: 13,941
Output tokens: 3,806
Total cost: $0.0555
Avg per request: 6970 in, 1903 out
=== NARRATIVE (before QC) ===
Okiku was born on a July morning in 1622 in a hamlet of Shinano Province, in the mountains where conifers stood close and fields clung to narrow flats. The Tokugawa shogunate ruled through the local domain and the village headman. Every household sat in the village register. The family’s temple kept the record that proved they were not Christians.

Her father, Shichibē, worked the small plots the household could get and carried loads for pay along the roads that cut through the valleys. When travelers and packhorse men passed, he went with them for days at a time. He kept a coil of rope by the door and a stitched straw hat that he mended until it would not hold together. Her mother, Otsune, rose first, banked the coals